In [2]:
# here, we convert population-1990-2020.csv into the proper timer series format and output population-1990-2020-final.csv

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('../../SharedData/dataset-generation/pop-1990-2020.csv')
#df.rename(columns={'Unnamed: 0': 'state'}, inplace=True)

d = {'state':[], 'date':[], 'monthly_population':[], 'yearly_population':[]}
new_df = pd.DataFrame(data=d)

In [3]:
# change state names to be abbreviations

full_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland",
    "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey",
    "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina",
    "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"
]

abbreviations = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]
name_dict = {full_names[i]:abbreviations[i] for i in range(len(full_names))}

df['state'] = df['state'].map(name_dict)

In [4]:
# loop through each year in 1960-2022 in given row in dataframe
# copy each year's energy data 12 times (one for each month) into 12 rows of new_df
# for a total of 12*63*50 entries

month_counter = 1   # acts like modulo 12 arithmetic
idx = 0             # keeps track of row index

for state in range(50):
    for year in range(1,len(df.iloc[state])-1):
        for month in range(12):
            new_df.loc[idx] = [df.iloc[state][0],df.columns[year]+'-'+'{:02d}'.format(month_counter),None,df.iloc[state][year]] 
            if month_counter < 12:                       
                if month_counter == 6:
                    new_df.loc[idx] = [df.iloc[state][0],df.columns[year]+'-'+'{:02d}'.format(month_counter),df.iloc[state][year],df.iloc[state][year]]   
                month_counter += 1
            else:
                month_counter = 1
            idx += 1

In [5]:
# get rid of commas
new_df['yearly_population'] = new_df['yearly_population'].str.replace(',', '').astype(int)
new_df['monthly_population'] = new_df['monthly_population'].apply(lambda x: int(str(x).replace(',', '')) if pd.notna(x) else x)
#new_df['monthly_population'] = new_df['monthly_population'].astype(int)

# linear interpolation to fill in months based on middle-of-year data

for state in abbreviations:
    new_df.loc[new_df.state == state,'monthly_population']=new_df[new_df.state == state].monthly_population.interpolate(method='linear')

In [6]:
# export new_df to a .csv
new_df.to_csv('../../SharedData/dataset-generation-final/population-1990-2020-final.csv')